In [ ]:
import pandas as pd
from collections import OrderedDict
import requests
import boto3

In [ ]:
breakdown_df = pd.ExcelFile("Copy of Breakdown Spreadsheet for Anastasia.xlsx")

## AB dataset
AB = pd.read_excel(breakdown_df, 'AB')
AB.columns = AB.iloc[0]

## CD dataset
CD = pd.read_excel(breakdown_df, 'CD')
CD.columns = CD.iloc[0]

## EF dataset
EF = pd.read_excel(breakdown_df, 'EF')
EF.columns = EF.iloc[1]

## GH dataset
GH = pd.read_excel(breakdown_df, 'GH')
GH.columns = GH.iloc[0]

## IJ dataset
IJ = pd.read_excel(breakdown_df, 'IJ')
IJ.columns = IJ.iloc[0]

## KL dataset
KL = pd.read_excel(breakdown_df, 'KL')
KL.columns = KL.iloc[0]

In [ ]:
def basic_clean(text):
    """
    A simple function to clean up the data. All the words that
    are not designated as a stop word is then lemmatized after
    encoding and basic regex parsing are performed.
    """
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')
    text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [ ]:
def convert_to_dict(dataset):
    cleaned_text = {}
    # child_context = list(dataset['Example: Child Context'].dropna())
    # print(len(child_context))
    # print(enumerate(child_context))
    # adult_response = list(replace(dataset['adult Response'], is.na(dataset['adult Response']), '')
    subdata = pd.DataFrame().assign(ChildContext = dataset['Example: Child Context'], AdultResponse = dataset['Example: adult Response'])
    subdata = subdata.dropna()
    child_context = list(subdata['ChildContext'])
    adult_response = list(subdata['AdultResponse'])
    for num in enumerate(child_context):
        cleaned_text.update({child_context[num[0]]:adult_response[num[0]]})
    return cleaned_text

datasets = [AB, CD, EF, GH, IJ, KL]
reformatted_data = pd.DataFrame(columns = ['text'])

In [ ]:
reformatted_data = {}
new_dat = {}
for dataset in datasets:
    print("dataset: ", len(dataset))
    new_dat = convert_to_dict(dataset)
    print("new dat: ", len(new_dat))
    reformatted_data.update(new_dat)
print(len(reformatted_data))

In [ ]:
formatted_data = pd.DataFrame(columns = ['text'])
text_column = []
for key in reformatted_data.keys():
    text_str = "<s>[INST] " + key + " [\INST] " + reformatted_data[key] + " <\s>"
    text_column.append(text_str)

formatted_data['text'] = text_column
print(formatted_data)

# formatted_data.to_csv("formatted_data.csv")


In [ ]:
old_grid_df = pd.ExcelFile("Grid CSV Examples and Tips.xlsx")
og_data = pd.read_excel(old_grid_df, "data-1608809159892")
og_data
og_data = pd.DataFrame().assign(ChildContext = og_data['example_sitation'], AdultResponse = og_data['example_Shadow_response'])
og_data = og_data.dropna()
og_data = og_data.drop_duplicates(subset=['ChildContext', 'AdultResponse'], keep="first")

In [ ]:
formatted_data = pd.DataFrame(columns = ['text'])
append_text = []
for i in range(len(list(og_data['ChildContext']))):
    text_str = "<s>[INST] " + list(og_data['ChildContext'])[i] + " [\INST] " + list(og_data['AdultResponse'])[i] + " <\s>"
    append_text.append(text_str)
text_column.extend(append_text)
formatted_data['text'] = text_column

In [ ]:
formatted_data['text']

In [ ]:
formatted_data.to_csv("formatted_data.csv")